<a href="https://colab.research.google.com/github/ebrahimElsharkwy/Sklearn/blob/master/Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#about this data
1.id: unique for new articles
2.title: the title of a new article
3.text: the text of the aricle
4.labels: real(0) or fake(1)

Importing *libraries*

In [2]:
import numpy as np
import pandas as pd
import re # use for searchnig about words in text and paragraph
from nltk.corpus import stopwords #uses to remove all stopwords from our dataset
from nltk.stem.porter import PorterStemmer #stem our word means it give us the root of word
from sklearn.feature_extraction.text import TfidfVectorizer #used to convert text to feature vectors 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Downloading Stopwords

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

What is this stop words ?

In [12]:
print(stopwords.words('english')) # and it can be 'arabic', 'french', 'spanish' and more languages

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Pre-processnig

In [13]:
#loading the dataset into our pandas dataframe
news_dataset = pd.read_csv('/content/train.csv')

In [16]:
# knowing the data shape
news_dataset.shape

(20800, 5)

In [17]:
# get first 5 rows of the data
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [18]:
# counting the number of missing values
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [19]:
# replacing the null values wuth empty string
news_dataset = news_dataset.fillna('')

In [20]:
# merging the author name and title
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [22]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


In [25]:
# separatnig new_data into  data & labels
X = news_dataset.drop('label', axis=1)
Y = news_dataset['label']
print(X, Y)

          id  ...                                            content
0          0  ...  Darrell Lucus House Dem Aide: We Didn’t Even S...
1          1  ...  Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2          2  ...  Consortiumnews.com Why the Truth Might Get You...
3          3  ...  Jessica Purkiss 15 Civilians Killed In Single ...
4          4  ...  Howard Portnoy Iranian woman jailed for fictio...
...      ...  ...                                                ...
20795  20795  ...  Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796  20796  ...  Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797  20797  ...  Michael J. de la Merced and Rachel Abrams Macy...
20798  20798  ...  Alex Ansary NATO, Russia To Hold Parallel Exer...
20799  20799  ...            David Swanson What Keeps the F-35 Alive

[20800 rows x 5 columns] 0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 2080

Stemming:

Stemming is the process of reducing a word to its root word

example:

actor, actress, acting ----> act

In [26]:
port_stem = PorterStemmer()

In [27]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]', ' ', content)# removve all data except letters
  stemmed_content = stemmed_content.lower() #converting all letters to lowercase
  stemmed_content = stemmed_content.split()#split words into list of letters
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]#get the root of the word and then remove stopwords
  stemmed_content = ' '.join(stemmed_content) #
  return stemmed_content

In [28]:
# then applying the function above into content column
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [29]:
print(news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [35]:
#separating data & label
X = news_dataset['content'].values
Y = news_dataset['label'].values
print(X)
X.shape

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


(20800,)

In [34]:
print(Y)
Y.shape

[1 0 1 ... 0 1 1]


(20800,)

In [36]:
# Converting textual data to neumerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [37]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

Splttnig data into trainnig and test data

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.33, stratify= Y, random_state = 2)

Training the model by logistic regression

In [40]:
LRModel = LogisticRegression()

In [41]:
LRModel.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Model Evaluation

In [44]:
# accuracy score for training data
train_predection = LRModel.predict(X_train)
train_accuracy = accuracy_score(train_predection, Y_train)
print('The Train Accuracy Socre is : ', train_accuracy)

The Train Accuracy Socre is :  0.9863662456946038


In [45]:
# accuracy score for test data
test_predection = LRModel.predict(X_test)
test_accuracy = accuracy_score(test_predection, Y_test)
print('The Test Accuracy Socre is : ', test_accuracy)

The Test Accuracy Socre is :  0.9723193473193473


Making Predictive System

In [51]:
X_new = X_test[1000]
new_prediction = LRModel.predict(X_new)
print(new_prediction)

if new_prediction == 0 :
  print('This news Is Real')
else:
  print('This news Is Fake')

[0]
This news Is Real


In [52]:
print(Y_test[1000])

0
